In [1]:
from keras.datasets import mnist
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os

In [2]:
# Training Parameters
batch_size = 64
epochs = 15

In [3]:
# loads the MNIST dataset
(x_train, y_train), (x_test, y_test)  = mnist.load_data()

In [4]:
# Lets store the number of rows and columns
img_rows = x_train[0].shape[0]
img_cols = x_train[1].shape[0]

In [5]:
# Getting our date in the right 'shape' needed for Keras
# We need to add a 4th dimenion to our date thereby changing our
# Our original image shape of (60000,28,28) to (60000,28,28,1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

In [6]:
x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [7]:
# store the shape of a single image 
input_shape = (img_rows, img_cols, 1)

# change our image type to float32 data type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize our data by changing the range from (0 to 255) to (0 to 1)
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [8]:
# Now we one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# Let's count the number columns in our hot encoded matrix 
print ("Number of Classes: " + str(y_test.shape[1]))

Number of Classes: 10


In [9]:
y_train.shape

(60000, 10)

In [10]:
num_classes = y_train.shape[1]
num_pixels = x_train.shape[1] * x_train.shape[2]

In [12]:
# create model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.Adadelta(),
              metrics = ['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                589888    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

In [19]:
checkpoint = ModelCheckpoint("Trained Models/MNIST_Checkpoint.h5",
                             monitor="val_accuracy",
                             mode="max",
                             save_best_only = True, save_weights_only=False, save_freq="epoch",
                             verbose=1)
callbacks = [checkpoint]   # create a list of callbacks

In [20]:
history = model.fit(x_train, y_train,
          batch_size = batch_size,
          epochs = epochs,
          verbose = 1,
          callbacks = callbacks,
          validation_data = (x_test, y_test))

Epoch 1/15
938/938 [==============================] - 109s 117ms/step - loss: 1.4688 - accuracy: 0.6748 - val_loss: 1.2254 - val_accuracy: 0.7952

Epoch 00001: val_accuracy improved from -inf to 0.79520, saving model to Trained Models\MNIST_Checkpoint.h5
Epoch 2/15
938/938 [==============================] - 102s 109ms/step - loss: 1.1790 - accuracy: 0.7090 - val_loss: 0.9459 - val_accuracy: 0.8232

Epoch 00002: val_accuracy improved from 0.79520 to 0.82320, saving model to Trained Models\MNIST_Checkpoint.h5
Epoch 3/15
938/938 [==============================] - 107s 114ms/step - loss: 0.9764 - accuracy: 0.7390 - val_loss: 0.7689 - val_accuracy: 0.8397

Epoch 00003: val_accuracy improved from 0.82320 to 0.83970, saving model to Trained Models\MNIST_Checkpoint.h5
Epoch 4/15
938/938 [==============================] - 105s 112ms/step - loss: 0.8488 - accuracy: 0.7624 - val_loss: 0.6595 - val_accuracy: 0.8514

Epoch 00004: val_accuracy improved from 0.83970 to 0.85140, saving model to Traine

KeyboardInterrupt: 

In [14]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/15
938/938 [==============================] - 224s 239ms/step - loss: 2.0890 - accuracy: 0.4245 - val_loss: 1.9718 - val_accuracy: 0.7093

Epoch 00001: val_loss improved from inf to 1.97180, saving model to //Trained Models\MNIST_Checkpoint.h5


OSError: Unable to create file (unable to open file: name = '//Trained Models\MNIST_Checkpoint.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

### Adding Multiple Call Backs & Early Stopping

We can use other call back methods to monitor our training process such as **Early Stopping**. Checkout the Keras documentation for more:
- https://keras.io/callbacks/

In [ ]:
from keras.callbacks import EarlyStopping

earlystop = EarlyStopping(monitor = 'val_loss', # value being monitored for improvement
                          min_delta = 1e-5, #Abs value and is the min change required before we stop
                          patience = 3, #Number of epochs we wait before stopping 
                          verbose = 1,
                          restore_best_weights = True) #keeps the best weigths once stopped

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

### We can attempt to run again to see if it worked!

In [ ]:
history = model.fit(x_train, y_train,
          batch_size=64,
          epochs=3,
          verbose=2,
          callbacks = callbacks,
          validation_data=(x_test, y_test))


score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### Another useful callback is Reducing our learning Rate on Plateau

We can avoid having our oscillate around the global minimum by attempting to reduce the Learn Rate by a certain fact. If no improvement is seen in our monitored metric (val_loss typically), we wait a certain number of epochs (patience) then this callback reduces the learning rate by a factor

In [ ]:
from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, verbose = 1, min_delta = 0.0001)